# Benchmarking high performance Pandas alternatives

In [ ]:
%%bash
pip3 -q install -U polars
pip3 -q install vaex
pip -q install datatable
#!pip3 install "git+https://github.com/h2oai/datatable.git"

In [ ]:
import pandas as pd
from time import time
import polars as pl
import vaex as vx
import datatable as dt
import plotly.express as px

In [ ]:
data_URL = "https://raw.githubusercontent.com/keitazoumana/Experimentation-Data/main/diabetes.csv"

original_data = pd.read_csv(data_URL)

# Duplicate each row 100000 times
benchmarking_df = original_data.loc[original_data.index.repeat(100000)]


In [ ]:
# Save the benchmarking data
file_name = "benchmarking_data.csv"

benchmarking_df.to_csv(file_name, index=False)

In [ ]:
def plot_metrics(list_exec_time, graph_title):

	df = pd.DataFrame(list_exec_time)

	fig = px.bar(df, x='library', y= 'execution_time', title=graph_title)

	fig.show()


def read_csv_with_time(library_name, file_name):

  final_time = 0

  start_time = time()

  if library_name.lower() == 'polars':
      df = pl.read_csv(file_name)

  elif library_name.lower() == 'pandas':
      df = pd.read_csv(file_name)

  elif library_name.lower() == 'vaex':
      df = vx.read_csv(file_name)

  elif library_name.lower() == 'datatable':
      df = dt.fread(file_name)

  else:
      raise ValueError("Invalid library name. Must be 'polars', 'pandas', 'vaex', or 'datatable'")

  end_time = time()

  final_time = end_time - start_time

  return {"library": library_name, "execution_time": final_time}, df




In [ ]:
pandas_time, pandas_df = read_csv_with_time('pandas', file_name)
polars_time, polars_df = read_csv_with_time('polars', file_name)
vaex_time, vaex_df = read_csv_with_time('vaex', file_name)
datatable_time, dt_df = read_csv_with_time('datatable', file_name)

In [ ]:
exec_times = [pandas_time, polars_time,
          	vaex_time, datatable_time]

def plot_metrics(list_exec_time, graph_title):

	[print(exec_time) for exec_time in list_exec_time]
	df = pd.DataFrame(exec_times)

	# Plot bar plot using Plotly Express
	fig = px.bar(df, x='library', y='execution_time', title=graph_title)
	fig.show()


plot_metrics(exec_times, "Read Execution Time Comparison")

{'library': 'pandas', 'execution_time': 35.07908916473389}
{'library': 'polars', 'execution_time': 6.041005373001099}
{'library': 'vaex', 'execution_time': 36.431522607803345}
{'library': 'datatable', 'execution_time': 4.4584901332855225}


## Data grouping

In [ ]:
#dt_df = dt.fread(file_name)

In [ ]:
from time import time

def group_data_with_time(library_name, df, column_name='Pregnancies'):

  start_time = time()

  if library_name.lower() == 'polars':
      df_grouped = df.groupby(column_name).first()

  elif library_name.lower() == 'vaex':
      df_grouped = df.groupby(column_name, agg='first')

  elif library_name.lower() == 'pandas':
      df_grouped = df.groupby(column_name).first()

  elif library_name.lower() == 'datatable':
      df_grouped = df[:, dt.first(dt.f[:]), dt.by(column_name)]

  else:
      raise ValueError("Invalid library name. Must be 'polars', 'vaex', or 'datatable'")

  end_time = time()

  final_time = end_time - start_time

  return {"library": library_name, "execution_time": final_time}

In [ ]:
pandas_time = group_data_with_time('pandas', pandas_df)
polars_time = group_data_with_time('polars', polars_df)
vaex_time = group_data_with_time('vaex', vaex_df)
datatable_time = group_data_with_time('datatable', dt_df)

<ipython-input-26-d5fdfb9c71ad>:8: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.



In [ ]:
exec_times = [pandas_time, polars_time,
          	vaex_time, datatable_time]


plot_metrics(exec_times, "Grouping Execution Time Comparison")

{'library': 'pandas', 'execution_time': 2.773515224456787}
{'library': 'polars', 'execution_time': 2.6934573650360107}
{'library': 'vaex', 'execution_time': 0.6478557586669922}
{'library': 'datatable', 'execution_time': 0.46595001220703125}


## Column sorting

In [ ]:
def sort_data_with_time(library_name, df, column_name='Pregnancies'):

  start_time = time()

  if library_name.lower() == 'polars':
      df_sorted = df.sort(column_name)
  elif library_name.lower() == 'vaex':
      df_sorted = df.sort(column_name)

  elif library_name.lower() == 'datatable':
      df_sorted = df.sort(column_name)

  elif library_name.lower() == 'pandas':
      df_sorted = pd.DataFrame(df).sort_values(column_name)
  else:
      raise ValueError("Invalid library name. Must be 'polars', 'vaex', 'datatable', or 'pandas'")

  end_time = time()

  final_time = end_time - start_time

  return {"library": library_name, "execution_time": final_time}

In [ ]:
pandas_time = sort_data_with_time('pandas', pandas_df)
polars_time = sort_data_with_time('polars', polars_df)
vaex_time = sort_data_with_time('vaex', vaex_df)
datatable_time = sort_data_with_time('datatable', dt_df)

In [ ]:
exec_times = [pandas_time, polars_time,
          	vaex_time, datatable_time]


plot_metrics(exec_times, "Sorting Execution Time Comparison")

{'library': 'pandas', 'execution_time': 6.735127687454224}
{'library': 'polars', 'execution_time': 4.119458436965942}
{'library': 'vaex', 'execution_time': 1.0371737480163574}
{'library': 'datatable', 'execution_time': 0.3971593379974365}


## Data Offloading

In [ ]:
def offload_data_with_time(library_name, df):

  start_time = time()

  if library_name.lower() == 'polars':
      array = df.to_numpy()

  elif library_name.lower() == 'vaex':
      array = df.to_pandas_df().values

  elif library_name.lower() == 'datatable':
      array = df.to_numpy()

  elif library_name.lower() == 'pandas':
      array = pd.DataFrame(df).values
  else:
      raise ValueError("Invalid library name. Must be 'polars', 'vaex', 'datatable', or 'pandas'")

  end_time = time()

  final_time = end_time - start_time

  return {"library": library_name, "execution_time": final_time}

In [ ]:
pandas_time = offload_data_with_time('pandas', pandas_df)
polars_time = offload_data_with_time('polars', polars_df)
vaex_time = offload_data_with_time('vaex', vaex_df)
datatable_time = offload_data_with_time('datatable', dt_df)

In [ ]:
exec_times = [pandas_time, polars_time,
          	vaex_time, datatable_time]


plot_metrics(exec_times, "Data offloading Execution Time Comparison")

{'library': 'pandas', 'execution_time': 1.8406445980072021}
{'library': 'polars', 'execution_time': 3.238591432571411}
{'library': 'vaex', 'execution_time': 6.945307970046997}
{'library': 'datatable', 'execution_time': 2.634136438369751}


## Benchmarking Memory Usage

In [ ]:
import tracemalloc as tm
import os

In [ ]:
list_memory_usage = []

In [ ]:
tm.start()
vaex_time = offload_data_with_time('vaex', vaex_df)
memory_usage = tm.get_traced_memory()
tm.stop()

list_memory_usage.append({
	'library': 'vaex',
	'memory_usage': memory_usage
})

In [ ]:
tm.start()
datatable_time = offload_data_with_time('datatable', dt_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
	'library': 'datatable',
	'memory_usage': memory_usage
})

In [ ]:
tm.start()
pandas_time = offload_data_with_time('pandas', pandas_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
	'library': 'pandas',
	'memory_usage': memory_usage
})

In [ ]:
tm.start()
polars_time = offload_data_with_time('polars', polars_df)
memory_usage = tm.get_traced_memory()

tm.stop()

list_memory_usage.append({
	'library': 'polars',
	'memory_usage': memory_usage
})

In [ ]:
def plot_memory_usage(list_memory_usage, graph_title='Memory Usage by Library'):

	df = pd.DataFrame(list_memory_usage)

	# separate the memory usage tuple into two columns: current_memory and peak_memory
	df[['current_memory', 'peak_memory']] = pd.DataFrame(df['memory_usage'].tolist(), index=df.index)

	# now we no longer need the memory_usage column
	df = df.drop(columns='memory_usage')


	# melt the DataFrame to make it suitable for grouped bar chart
	df_melted = df.melt(id_vars='library', var_name='memory_type', value_name='memory')

	# create the grouped bar chart
	fig = px.bar(df_melted, x='library', y='memory', color='memory_type', barmode='group',
             	labels={'memory':'Memory Usage (bytes)', 'library':'Library', 'memory_type':'Memory Type'},
             	title=graph_title)


	fig.update_layout(yaxis_type="log")
	fig.show()



In [ ]:
plot_memory_usage(list_memory_usage)